In [1]:
import pandas as pd
import yaml
import pykakasi as pk

In [2]:
original_label = './data/original/jsut-label-master/text_kana/basic5000.yaml'
with open(original_label, mode='r') as yamlfile:
        label_info = yaml.safe_load(yamlfile)

In [3]:
tgt = 'char'

In [4]:
kks = pk.kakasi()

In [5]:
%%time
filenames  = []
label_char = []
label_kana = []
label_phone = []
label_roman = []
for i in range(5000):
    # 発話ID
    filename = 'BASIC5000_%04d' % (i+1)
    
    filenames.append(f'./data/wav/{filename}.wav')

    # 発話ID が label_info に含まれない場合はエラー
    if not filename in label_info:
        print('Error: %s is not in %s' % (filename, original_label))
        exit()

    # キャラクターラベル情報を取得
    chars = label_info[filename]['text_level2']
    # '、'と'。'を除去
    chars = chars.replace('、', '')
    chars = chars.replace('。', '')

    # かなラベル情報を取得
    kanas = label_info[filename]['kana_level3']
    # '、'を除去
    kanas = kanas.replace('、', '')

    # 音素ラベル情報を取得
    phones = label_info[filename]['phone_level3']
    
    result = kks.convert(kanas)
    

    # キャラクターラベルファイルへ，1文字ずつスペース区切りで書き込む
    # (' '.join(list) は，リストの各要素にスペースを挟んで，1文にする)
    label_char.append(' '.join(chars))

    # かなラベルファイルへ，1文字ずつスペース区切りで書き込む
    label_kana.append(' '.join(kanas))

    # 音素ラベルは，'-'をスペースに置換して書き込む
    label_phone.append(' '.join(phones).replace('-',' '))
    
    tmp = []
    for j in range(len(result)):
        tmp.append(result[j]['hepburn'].lower().replace("'",' '))
        
    label_roman.append(' '.join(tmp))

CPU times: user 1.68 s, sys: 8.4 ms, total: 1.68 s
Wall time: 1.68 s


In [6]:
df = pd.DataFrame(
    data={
            'filepath': filenames,
            'char': label_char, 
            'hira': label_kana,
            'phone': label_phone,
            'roman': label_roman
            }
)
df.head()

,filepath,char,hira,phone,roman
0,./data/wav/BASIC5000_0001.wav,水 を マ レ ー シ ア か ら 買 わ な く て は な ら な い の で す,み ず を ま れ ー し あ か ら か わ な く て わ な ら な い の で す,m i z u o m a r e e s h ...,mizuwomareeshiakarakawanakutewanaranainodesu
1,./data/wav/BASIC5000_0002.wav,木 曜 日 停 戦 会 談 は 何 の 進 展 も な い ま ま 終 了 し ま し た,も く よ ー び て ー せ ん か い だ ん わ な ん の し ん て ん も な ...,m o k u y o o b i p a u t ...,mokuyoobiteesenkaidanwanannoshintenmonaimamash...
2,./data/wav/BASIC5000_0003.wav,上 院 議 員 は 私 が デ ー タ を ゆ が め た と 告 発 し た,じ ょ ー い ん ぎ ー ん わ わ た し が で ー た を ゆ が め た と こ ...,j o o i N g i i N w a p ...,jooingiinwawatashigadeetawoyugametatokokuhatsu...
3,./data/wav/BASIC5000_0004.wav,１ 週 間 し て そ の ニ ュ ー ス は 本 当 に な っ た,い っ し ゅ ー か ん し て そ の に ゅ ー す わ ほ ん と ー に な っ た,i c l s h u u k a N s h i ...,isshuukanshitesononyuusuwahontooninatta
4,./data/wav/BASIC5000_0005.wav,血 圧 は 健 康 の パ ロ メ ー タ ー と し て 重 要 で あ る,け つ あ つ わ け ん こ ー の ぱ ろ め ー た ー と し て じ ゅ ー よ ...,k e t s u a t s u w a p a u ...,ketsuatsuwakenkoonoparomeetaatoshitejuuyoodearu


In [7]:
val = df[:250]
train = df[250:]
len(train)

4750

In [8]:
train.head()

,filepath,char,hira,phone,roman
250,./data/wav/BASIC5000_0251.wav,演 奏 会 の 切 符 は 当 事 務 所 で 販 売 し て い ま す,え ん そ ー か い の き っ ぷ わ と ー じ む し ょ で は ん ば い し ...,e N s o o k a i n o k i ...,ensookainokippuwatoojimushodehanbaishiteimasu
251,./data/wav/BASIC5000_0252.wav,市 場 価 格 の 暴 落 で 一 文 無 し に な っ て し ま っ た,し じ ょ ー か か く の ぼ ー ら く で い ち も ん な し に な っ て ...,s h i j o o k a k a k u ...,shijookakakunoboorakudeichimonnashininatteshim...
252,./data/wav/BASIC5000_0253.wav,女 と い う も の は 何 で も お 金 に 換 算 し て 考 え る,お ん な と ゆ ー も の わ な ん で も お か ね に か ん ざ ん し て ...,o N n a t o y u u m o n ...,onnatoyuumonowanandemookanenikanzanshitekangaeru
253,./data/wav/BASIC5000_0254.wav,女 は テ ー ブ ル の ナ イ フ に 手 を 伸 ば し た,お ん な わ て ー ぶ る の な い ふ に て を の ば し た,o N n a w a t e e b u r ...,onnawateeburunonaifunitewonobashita
254,./data/wav/BASIC5000_0255.wav,女 の 子 は い い 服 を 見 せ び ら か す の が 好 き だ,お ん な の こ わ い い ふ く を み せ び ら か す の が す き だ,o N n a n o k o w a p a u ...,onnanokowaiifukuwomisebirakasunogasukida


In [9]:
val.head()

,filepath,char,hira,phone,roman
0,./data/wav/BASIC5000_0001.wav,水 を マ レ ー シ ア か ら 買 わ な く て は な ら な い の で す,み ず を ま れ ー し あ か ら か わ な く て わ な ら な い の で す,m i z u o m a r e e s h ...,mizuwomareeshiakarakawanakutewanaranainodesu
1,./data/wav/BASIC5000_0002.wav,木 曜 日 停 戦 会 談 は 何 の 進 展 も な い ま ま 終 了 し ま し た,も く よ ー び て ー せ ん か い だ ん わ な ん の し ん て ん も な ...,m o k u y o o b i p a u t ...,mokuyoobiteesenkaidanwanannoshintenmonaimamash...
2,./data/wav/BASIC5000_0003.wav,上 院 議 員 は 私 が デ ー タ を ゆ が め た と 告 発 し た,じ ょ ー い ん ぎ ー ん わ わ た し が で ー た を ゆ が め た と こ ...,j o o i N g i i N w a p ...,jooingiinwawatashigadeetawoyugametatokokuhatsu...
3,./data/wav/BASIC5000_0004.wav,１ 週 間 し て そ の ニ ュ ー ス は 本 当 に な っ た,い っ し ゅ ー か ん し て そ の に ゅ ー す わ ほ ん と ー に な っ た,i c l s h u u k a N s h i ...,isshuukanshitesononyuusuwahontooninatta
4,./data/wav/BASIC5000_0005.wav,血 圧 は 健 康 の パ ロ メ ー タ ー と し て 重 要 で あ る,け つ あ つ わ け ん こ ー の ぱ ろ め ー た ー と し て じ ゅ ー よ ...,k e t s u a t s u w a p a u ...,ketsuatsuwakenkoonoparomeetaatoshitejuuyoodearu


In [10]:
vocab_val = []
for i in range(len(val)):
    for j in range(len(val[tgt].iloc[i])):
        vocab_val.append(val[tgt].iloc[i][j:j+1])
        if "'"==val[tgt].iloc[i][j:j+1]: print(val[tgt].iloc[i])
        
vocab_train = []
for i in range(len(train)):
    for j in range(len(train[tgt].iloc[i])):
        vocab_train.append(train[tgt].iloc[i][j:j+1])
        

vocab_list = sorted(list(set(set(vocab_train) | set(vocab_val))))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict


{' ': 0,
 '-': 1,
 '0': 2,
 '1': 3,
 '5': 4,
 'C': 5,
 'P': 6,
 '々': 7,
 'ぁ': 8,
 'あ': 9,
 'い': 10,
 'う': 11,
 'え': 12,
 'ぉ': 13,
 'お': 14,
 'か': 15,
 'が': 16,
 'き': 17,
 'ぎ': 18,
 'く': 19,
 'ぐ': 20,
 'け': 21,
 'げ': 22,
 'こ': 23,
 'ご': 24,
 'さ': 25,
 'ざ': 26,
 'し': 27,
 'じ': 28,
 'す': 29,
 'ず': 30,
 'せ': 31,
 'ぜ': 32,
 'そ': 33,
 'ぞ': 34,
 'た': 35,
 'だ': 36,
 'ち': 37,
 'っ': 38,
 'つ': 39,
 'づ': 40,
 'て': 41,
 'で': 42,
 'と': 43,
 'ど': 44,
 'な': 45,
 'に': 46,
 'ぬ': 47,
 'ね': 48,
 'の': 49,
 'は': 50,
 'ば': 51,
 'ぱ': 52,
 'ひ': 53,
 'び': 54,
 'ぴ': 55,
 'ふ': 56,
 'ぶ': 57,
 'ぷ': 58,
 'へ': 59,
 'べ': 60,
 'ぺ': 61,
 'ほ': 62,
 'ぼ': 63,
 'ぽ': 64,
 'ま': 65,
 'み': 66,
 'む': 67,
 'め': 68,
 'も': 69,
 'ゃ': 70,
 'や': 71,
 'ゅ': 72,
 'ゆ': 73,
 'ょ': 74,
 'よ': 75,
 'ら': 76,
 'り': 77,
 'る': 78,
 'れ': 79,
 'ろ': 80,
 'わ': 81,
 'を': 82,
 'ん': 83,
 'ァ': 84,
 'ア': 85,
 'ィ': 86,
 'イ': 87,
 'ゥ': 88,
 'ウ': 89,
 'ェ': 90,
 'エ': 91,
 'ォ': 92,
 'オ': 93,
 'カ': 94,
 'ガ': 95,
 'キ': 96,
 'ギ': 97,
 'ク': 98,
 'グ': 99,
 'ケ': 100,

In [11]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

2699

In [12]:
import json
with open(f'vocab_{tgt}.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [13]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer(f"./vocab_{tgt}.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
tokenizer

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


PreTrainedTokenizer(name_or_path='', vocab_size=2699, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'})

In [14]:
train.to_csv(f'train_{tgt}.csv')
val.to_csv(f'val_{tgt}.csv')